# **0 模型构建思路及调优过程**

# **0.1.1 完整算法结构框图**

本项目采用faster rcnn 的swin transformer和r101分别训练，然后将两个模型的结果进行融合

swin transformer结构框图
![](https://ai-studio-static-online.cdn.bcebos.com/26d3a837358347478a3d648fa03869c48b4491cbdc1549e6b6e7aeda28cdefb2)

r101结构框图
![](https://ai-studio-static-online.cdn.bcebos.com/aa90531cfc6f4c788c06493376759769e6759af6dc6a4753ae72c1c70efc0b39)

# 
# **0.1.2 思路步骤详述**

项目首先进行数据预处理，然后分别利用faster rcnn的swin transformer和r101架构进行训练，最后将模型预测结果利用WBF进行融合。
NMS和soft-NMS都排除了一些框，但是WBF利用了所有框的信息。它可以解决某些情况下，模型预测的框不准确。NMS将只能留下一个不准确的框，而WBF将使用来自3个框的信息来修复它，如下图所示，红色的为预测，蓝色的为真值。

![](https://ai-studio-static-online.cdn.bcebos.com/8efbab1a3a4348649baf6c24984d60eff04b2bd73916402db9804e4c79e136ac)

# **0.1.3 代码组织结构介绍**

项目代码目录

0 模型构建思路及调优过程

1 比赛介绍

2 数据介绍

3 环境准备

4 数据准备

5 数据增强

6 训练模型

7 检测结果

8 融合结果

# **0.2 数据增强/清晰策略**

项目尝试进行数据增强，通过实验结果分析，增强效果不明显，故最后没有使用数据增强


# **0.3 调参优化策略**
项目首先将train数据集进行训练集：验证集=9：1进行划分，通过loss以及mAP调整学习率和衰减轮次等参数。
最后调整到结果较好时，保持训练超参数不变，将所有train数据全部用于训练

# **0.4训练脚本/代码**
见6 训练模型，训练日志保存在/home/aistudio/log

# **0.5测试脚本/代码**
见7 检测结果，测试集为比赛测试数据，无标签，测试结果保存在/home/aistudio/Csv

# **1 比赛介绍**
缺陷检测技术广泛应用于工业场景，比如汽车制造中的车身表面缺陷检测，零件外观缺陷检测，工件裂纹检测等。其中，金属表面缺陷识别技术的应用可以在生产及制造阶段的质量控制方面发挥重要作用。

本次比赛为图像目标识别比赛，要求参赛选手基于给定图像建立模型，识别出钢铁表面出现缺陷的位置，并给出锚点框的坐标，同时对不同的缺陷进行分类。通过本次比赛，我们希望看到各种泛化性能更好且更稳定的钢铁表面缺陷识别模型，您的工作也将为传统钢铁产业生产效率的提升做出重要贡献。

# **2 数据介绍**
东北大学宋克臣教授提供的可开放使用的表面缺陷检测数据集

# **3 环境准备**
为实验配置相应的环境

In [2]:
!pip config list
# 安装paddlex 用于拆分数据集
# 升级pip
!pip install --upgrade pip -i https://mirror.baidu.com/pypi/simple
!pip install "paddlex>2.0.0" -i https://mirror.baidu.com/pypi/simple 
# 统计数据
!pip install scikit-image
!pip install lxml
# # 下载PaddleDetection 
%cd /home/aistudio/work
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git -b release/2.3 
# 安装其它依赖
!pip install -r /home/aistudio/work/PaddleDetection/requirements.txt  
# 临时环境安装
!pip install pycocotools -i https://mirror.baidu.com/pypi/simple
!pip install lap -i https://mirror.baidu.com/pypi/simple
!pip install ensemble-boxes   

global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'
Looking in indexes: https://mirror.baidu.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Looking in indexes: https://mirror.baidu.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 9.2 MB/s eta 0:00:00:0

# **4 数据准备**
分析实验数据
划分训练集测试集

In [4]:
# 解压文件并移除多余的目录
! unzip /home/aistudio/data/data105746/train.zip -d /home/aistudio/data/steel
!rm -r /home/aistudio/data/steel/__MACOSX
! unzip /home/aistudio/data/data105747/test.zip -d /home/aistudio/data/steel
!rm -r /home/aistudio/data/steel/__MACOSX

# 修改文件名字 JPEGImages  Annotations
!mv /home/aistudio/data/steel/train/ANNOTATIONS  /home/aistudio/data/steel/train/Annotations
!mv /home/aistudio/data/steel/train/IMAGES  /home/aistudio/data/steel/train/JPEGImages

  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._84.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/608.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._608.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/90.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._90.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/620.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._620.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/146.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._146.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/152.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._152.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATIONS/634.xml  
  inflating: /home/aistudio/data/steel/__MACOSX/train/ANNOTATIONS/._634.xml  
  inflating: /home/aistudio/data/steel/train/ANNOTATION

In [3]:
# 分析实验数据
!python /home/aistudio/strongData/sample_num.py  

scratches:420个
crazing:537个
patches:679个
inclusion:788个
rolled-in_scale:493个
pitted_surface:339个
信息统计算完毕。


In [5]:
#使用paddleX拆分数据集
!paddlex --split_dataset --format VOC --dataset_dir /home/aistudio/data/steel/train --val_value 0.001 --test_value 0.0
%cd /home/aistudio/work/PaddleDetection/
#转换train
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/train_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_train.json

#转换test
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/val_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_val.json

!rm -r /home/aistudio/data/steel/train/Annotations/*
!mv /home/aistudio/data/steel/train/*.json /home/aistudio/data/steel/train/Annotations/

[12-28 12:06:13 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format VOC --dataset_dir /home/aistudio/data/steel/train --val_value 0.001 --test_value 0.0
[12-28 12:06:13 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/py

# **5 数据增强**
经过实验对比 增强前和增强后好像并没有提高太多的效果 所以这里我最后还是没用
可能是我增强的方式有问题 你们可以试试

In [ ]:
# # 数据增强 基准数扩充2倍
# !python /home/aistudio/strongData/strong.py
# # 合并增强数据
# !mv /home/aistudio/data/steel/train/AugAnnotations/* /home/aistudio/data/steel/train/Annotations
# !mv /home/aistudio/data/steel/train/AugJPEGImages/* /home/aistudio/data/steel/train/JPEGImages
# # 清除空目录
# !rm -rf /home/aistudio/data/steel/train/AugAnnotations
# !rm -rf /home/aistudio/data/steel/train/AugJPEGImages

# **6 训练模型**
经过多次实验 观察到faster rcnn的Swin和r101 效果是最好的 训练这两个模型
注意训练前配置好dataset的路径

In [ ]:
# 训练
!python tools/train.py -c /home/aistudio/work/PaddleDetection/configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml --use_vdl=true --vdl_log_dir=vdl_dir/scalar --eval

 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.700
[12/26 13:17:21] ppdet.engine INFO: Total sample number: 1, averge FPS: 2.3917784440767003
[12/26 13:17:21] ppdet.engine INFO: Best test bbox ap is 0.600.
[12/26 13:17:21] ppdet.engine INFO: Epoch: [15] [  0/349] learning_rate: 0.000100 loss_rpn_cls: 0.009663 loss_rpn_reg: 0.019303 loss_bbox_cls: 0.116640 loss_bbox_reg: 0.201563 loss: 0.355428 eta: 1:12:30 batch_cost: 0.5687 data_cost: 0.0005 ips: 7.0331 images/s
[12/26 13:17:33] ppdet.engine INFO: Epoch: [15] [ 20/349] learning_rate: 0.000100 loss_rpn_cls: 0.008259 loss_rpn_reg: 0.020569 loss_bbox_cls: 0.125067 loss_bbox_reg: 0.180972 loss: 0.331293 eta: 1:12:17 batch_cost: 0.5778 data_cost: 0.0003 ips: 6.9228 images/s
[12/26 13:17:45] ppdet.engine INFO: Epoch: [15] [ 40/349] learning_rate: 0.0001

In [11]:
# 训练
!python tools/train.py -c /home/aistudio/work/PaddleDetection/configs/faster_rcnn/faster_rcnn_r101_vd_fpn_2x_coco.yml --use_vdl=true --vdl_log_dir=vdl_dir/scalar --eval

[12/26 19:00:33] ppdet.engine INFO: Epoch: [60] [ 220/1399] learning_rate: 0.000008 loss_rpn_cls: 0.001342 loss_rpn_reg: 0.008886 loss_bbox_cls: 0.067110 loss_bbox_reg: 0.123854 loss: 0.217028 eta: 0:31:43 batch_cost: 0.1483 data_cost: 0.0009 ips: 6.7425 images/s
[12/26 19:00:36] ppdet.engine INFO: Epoch: [60] [ 240/1399] learning_rate: 0.000008 loss_rpn_cls: 0.002389 loss_rpn_reg: 0.010646 loss_bbox_cls: 0.083612 loss_bbox_reg: 0.150125 loss: 0.246737 eta: 0:31:41 batch_cost: 0.1489 data_cost: 0.0006 ips: 6.7156 images/s
[12/26 19:00:38] ppdet.engine INFO: Epoch: [60] [ 260/1399] learning_rate: 0.000008 loss_rpn_cls: 0.003432 loss_rpn_reg: 0.009255 loss_bbox_cls: 0.055778 loss_bbox_reg: 0.100388 loss: 0.183364 eta: 0:31:38 batch_cost: 0.1376 data_cost: 0.0003 ips: 7.2653 images/s
[12/26 19:00:41] ppdet.engine INFO: Epoch: [60] [ 280/1399] learning_rate: 0.000008 loss_rpn_cls: 0.001860 loss_rpn_reg: 0.013090 loss_bbox_cls: 0.065871 loss_bbox_reg: 0.109410 loss: 0.218547 eta: 0:31:35 ba

# **7 检测结果**

In [6]:
# Swin
%cd /home/aistudio/work/PaddleDetection

!python tools/infer.py -c  configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml \
-o weights=/home/aistudio/work/PaddleDetection/output/faster_rcnn_swin_tiny_fpn_3x_coco/model_final \
--infer_dir=/home/aistudio/data/steel/test/IMAGES/ \
--output_dir=/home/aistudio/data/steel/infer_output\
--draw_threshold=0.0000001 --save_txt=True

/home/aistudio/work/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W1228 12:06:37.945428  2931 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W1228 12:06:37.950759  2931 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[12/28 12:06:42] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/work/PaddleDetection/output/faster_rcnn_swin_tiny_fpn_3x_coco/model_final.pdparams
[12/28 12:06:42] train INFO: Found 400 inference images in total.
loading annotations into memory...
Done (t=0.00s)
cre

In [7]:
# 将Swin模型的检测结果保存为csv文件
import csv
import os
headers = ['image_id','bbox','category_id','confidence']
classList = ['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
rows = []

rootdir = '/home/aistudio/data/steel/infer_output'
lists = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(lists)):
       path = os.path.join(rootdir,lists[i])
       if os.path.isfile(path) and path.endswith('txt'):
           txtFile = open(path)
           print(path)
           result = txtFile.readlines()
           for r in result:
               ls = r.split(' ')
               Cls = ls[0]
               sco = float(ls[1])
              #  if sco < 0.0001:
              #    continue;
               xmin = float(ls[2])
               ymin = float(ls[3])
               w = float(ls[4])
               h = float(ls[5])
               xmax = xmin+w
               ymax = ymin+h
               clsID = classList.index(Cls)
               imgID = lists[i][:-4]
               row = [imgID,[xmin,ymin,xmax,ymax],clsID,sco]
               rows.append(row)
with open('/home/aistudio/Csv/比赛提交/Swin/submission.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

import pandas as pd
datafile = pd.read_csv('/home/aistudio/Csv/比赛提交/Swin/submission.csv')
# 按照列值排序
data = datafile.sort_values(by="image_id", ascending=True)
data.to_csv('/home/aistudio/Csv/比赛提交/Swin/submission_final.csv', mode='a+', index=False)

/home/aistudio/data/steel/infer_output/1580.txt
/home/aistudio/data/steel/infer_output/1576.txt
/home/aistudio/data/steel/infer_output/1444.txt
/home/aistudio/data/steel/infer_output/1524.txt
/home/aistudio/data/steel/infer_output/1772.txt
/home/aistudio/data/steel/infer_output/1522.txt
/home/aistudio/data/steel/infer_output/1590.txt
/home/aistudio/data/steel/infer_output/1636.txt
/home/aistudio/data/steel/infer_output/1427.txt
/home/aistudio/data/steel/infer_output/1490.txt
/home/aistudio/data/steel/infer_output/1644.txt
/home/aistudio/data/steel/infer_output/1657.txt
/home/aistudio/data/steel/infer_output/1561.txt
/home/aistudio/data/steel/infer_output/1441.txt
/home/aistudio/data/steel/infer_output/1796.txt
/home/aistudio/data/steel/infer_output/1458.txt
/home/aistudio/data/steel/infer_output/1525.txt
/home/aistudio/data/steel/infer_output/1646.txt
/home/aistudio/data/steel/infer_output/1743.txt
/home/aistudio/data/steel/infer_output/1609.txt
/home/aistudio/data/steel/infer_output/1

In [8]:
# r101
%cd /home/aistudio/work/PaddleDetection

!python tools/infer.py -c  configs/faster_rcnn/faster_rcnn_r101_vd_fpn_2x_coco.yml \
-o weights=/home/aistudio/work/PaddleDetection/output/faster_rcnn_r101_vd_fpn_2x_coco/model_final \
--infer_dir=/home/aistudio/data/steel/test/IMAGES/ \
--output_dir=/home/aistudio/data/steel/infer_output\
--draw_threshold=0.0000001 --save_txt=True

/home/aistudio/work/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W1228 12:08:18.296759  3209 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W1228 12:08:18.302073  3209 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[12/28 12:08:21] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/work/PaddleDetection/output/faster_rcnn_r101_vd_fpn_2x_coco/model_final.pdparams
[12/28 12:08:21] train INFO: Found 400 inference images in total.
loading annotations into memory...
Done (t=0.00s)
creat

In [9]:
# 将r101模型的检测结果保存为csv文件
import csv
import os
headers = ['image_id','bbox','category_id','confidence']
classList = ['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
rows = []

rootdir = '/home/aistudio/data/steel/infer_output'
lists = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(lists)):
       path = os.path.join(rootdir,lists[i])
       if os.path.isfile(path) and path.endswith('txt'):
           txtFile = open(path)
           print(path)
           result = txtFile.readlines()
           for r in result:
               ls = r.split(' ')
               Cls = ls[0]
               sco = float(ls[1])
              #  if sco < 0.0001:
              #    continue;
               xmin = float(ls[2])
               ymin = float(ls[3])
               w = float(ls[4])
               h = float(ls[5])
               xmax = xmin+w
               ymax = ymin+h
               clsID = classList.index(Cls)
               imgID = lists[i][:-4]
               row = [imgID,[xmin,ymin,xmax,ymax],clsID,sco]
               rows.append(row)
with open('/home/aistudio/Csv/比赛提交/r101/submission.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

import pandas as pd
datafile = pd.read_csv('/home/aistudio/Csv/比赛提交/r101/submission.csv')
# 按照列值排序
data = datafile.sort_values(by="image_id", ascending=True)
data.to_csv('/home/aistudio/Csv/比赛提交/r101/submission_final.csv', mode='a+', index=False)

/home/aistudio/data/steel/infer_output/1580.txt
/home/aistudio/data/steel/infer_output/1576.txt
/home/aistudio/data/steel/infer_output/1444.txt
/home/aistudio/data/steel/infer_output/1524.txt
/home/aistudio/data/steel/infer_output/1772.txt
/home/aistudio/data/steel/infer_output/1522.txt
/home/aistudio/data/steel/infer_output/1590.txt
/home/aistudio/data/steel/infer_output/1636.txt
/home/aistudio/data/steel/infer_output/1427.txt
/home/aistudio/data/steel/infer_output/1490.txt
/home/aistudio/data/steel/infer_output/1644.txt
/home/aistudio/data/steel/infer_output/1657.txt
/home/aistudio/data/steel/infer_output/1561.txt
/home/aistudio/data/steel/infer_output/1441.txt
/home/aistudio/data/steel/infer_output/1796.txt
/home/aistudio/data/steel/infer_output/1458.txt
/home/aistudio/data/steel/infer_output/1525.txt
/home/aistudio/data/steel/infer_output/1646.txt
/home/aistudio/data/steel/infer_output/1743.txt
/home/aistudio/data/steel/infer_output/1609.txt
/home/aistudio/data/steel/infer_output/1

# **8 融合结果**
将两组结果融合

In [10]:
# 融合两个模型
import numpy as np
from ensemble_boxes import *
import pandas as pd
import csv

# 定义融合两个模型结果的函数
def wbf_2models(filename1, filename2, weights, iou_thr):
    df1=pd.read_csv(filename1)
    df2=pd.read_csv(filename2)

    box1=[]
    box2=[]
    label1=[]
    label2=[]
    score1=[]
    score2=[]
    j=0
    k=0
    for i in range(400):
        box1.append([])
        box2.append([])
        label1.append([])
        label2.append([])
        score1.append([])
        score2.append([])

    # 将结果文件中的数据放入列表中，方便处理
    for id1 in range(1400,1800):    
        while j<len(df1) and df1['image_id'][j]==id1:
            box1[id1-1400].append(eval(df1['bbox'][j]))
            label1[id1-1400].append(df1['category_id'][j])
            score1[id1-1400].append(df1['confidence'][j])
            j+=1
        while k<len(df2) and df2['image_id'][k]==id1:
            box2[id1-1400].append(eval(df2['bbox'][k]))
            label2[id1-1400].append(df2['category_id'][k])
            score2[id1-1400].append(df2['confidence'][k])
            k+=1

    for l in range(len(box1)):  
        for i in range(len(box1[l])):
            box1[l][i][0]=box1[l][i][0]/200.0         # 归一化处理
            box1[l][i][1]=box1[l][i][1]/200.0
            box1[l][i][2]=box1[l][i][2]/200.0
            box1[l][i][3]=box1[l][i][3]/200.0
        for i in range(len(box2[l])):
            box2[l][i][0]=box2[l][i][0]/200.0
            box2[l][i][1]=box2[l][i][1]/200.0
            box2[l][i][2]=box2[l][i][2]/200.0
            box2[l][i][3]=box2[l][i][3]/200.0

    boxes=[]
    scores=[]
    labels=[]
     # 进行结果融合
    for i in range(400):
        label_list=[label1[i],label2[i]]
        box_list=[box1[i],box2[i]]

        score_list=[score1[i],score2[i]]
        box, score, label = weighted_boxes_fusion(box_list, score_list, label_list, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
    
        boxes.append(list(box))
        scores.append(list(score))
        labels.append(list(label.astype(int)))

    data=[]
    for i in range(400):
        for j in range(len(labels[i])):
            boxes[i][j][0]*=200.0      # 反归一化
            boxes[i][j][1]*=200.0
            boxes[i][j][2]*=200.0
            boxes[i][j][3]*=200.0
            boxes[i][j]=[boxes[i][j][0],boxes[i][j][1],boxes[i][j][2],boxes[i][j][3]]
            data.append([i+1400,boxes[i][j],labels[i][j],scores[i][j]])
    # 将融合后的结果写入新的文件
    with open("/home/aistudio/Csv/比赛提交/Swin+r101/submission.csv",'w',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(['image_id','bbox','category_id','confidence'])
        writer.writerows(data)

In [11]:
submission1 = '/home/aistudio/Csv/比赛提交/Swin/submission_final.csv'    # 文件路径
submission2 = '/home/aistudio/Csv/比赛提交/r101/submission_final.csv'
weights=[3,1]
iou_thr=0.7
wbf_2models(submission1, submission2, weights, iou_thr)

最终融合的结果保存在 /home/aistudio/Csv/比赛提交/Swin+r101

# **0 模型构建思路及调优过程**

**0.1.1 完整算法结构框图**

本项目采用faster rcnn 的swin transformer和r101分别训练，然后将两个模型的结果进行融合

swin transformer结构框图
![](https://ai-studio-static-online.cdn.bcebos.com/26d3a837358347478a3d648fa03869c48b4491cbdc1549e6b6e7aeda28cdefb2)

r101结构框图
![](https://ai-studio-static-online.cdn.bcebos.com/aa90531cfc6f4c788c06493376759769e6759af6dc6a4753ae72c1c70efc0b39)


**0.1.2 思路步骤详述**

项目首先进行数据预处理，然后分别利用faster rcnn的swin transformer和r101架构进行训练，最后将模型预测结果利用WBF进行融合。
NMS和soft-NMS都排除了一些框，但是WBF利用了所有框的信息。它可以解决某些情况下，模型预测的框不准确。NMS将只能留下一个不准确的框，而WBF将使用来自3个框的信息来修复它，如下图所示，红色的为预测，蓝色的为真值。

![](https://ai-studio-static-online.cdn.bcebos.com/8efbab1a3a4348649baf6c24984d60eff04b2bd73916402db9804e4c79e136ac)

**0.1.3 代码组织结构介绍**
项目代码目录
0 模型构建思路及调优过程
1 比赛介绍
2 数据介绍
3 环境准备
4 数据准备
5 数据增强
6 训练模型
7 检测结果
8 融合结果
